In [ ]:
import numpy as np 
import pandas as pd 
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [ ]:
alarm1 = pd.read_csv('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_1/alarm.csv')
causal1 = np.load('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_1/causal_prior.npy' , allow_pickle = True)

alarm2 = pd.read_csv('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_2/alarm.csv')
causal2 = np.load('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_2/causal_prior.npy' , allow_pickle = True)

alarm3 = pd.read_csv('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_3/alarm.csv')
causal3 = np.load('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_3/causal_prior.npy' , allow_pickle = True)

alarm4 = pd.read_csv('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_4/alarm.csv')
causal4 = np.load('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_4/causal_prior.npy' , allow_pickle = True)

rca1 = pd.read_csv('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_1/rca_prior.csv')
topology1 = np.load('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_1/topology.npy' , allow_pickle = True)
rca2 = pd.read_csv('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_2/rca_prior.csv')
topology2 = np.load('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_2/topology.npy' , allow_pickle = True)
rca3 = pd.read_csv('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_3/rca_prior.csv')
topology3 = np.load('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_3/topology.npy' , allow_pickle = True)

In [ ]:
class LogisticRegressionModel:
    def __init__(self, learning_rate=0.01, epochs=1000):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.weights = None
        self.bias = None

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def train(self, X_train, y_train):
        num_samples, num_features = X_train.shape
        self.weights = np.zeros(num_features)
        self.bias = np.zeros(y_train.shape[1])  # Initialize bias for each unique alarm ID

        # Batch Gradient Descent
        for _ in range(self.epochs):
            model = np.dot(X_train, self.weights) + self.bias
            predictions = self.sigmoid(model)

            # Compute Gradients
            dw = (1 / num_samples) * np.dot(X_train.T, (predictions - y_train))
            db = (1 / num_samples) * np.sum(predictions - y_train, axis=0)

            # Update Parameters
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X_test, n_unique_alarms):
        model = np.dot(X_test, self.weights) + self.bias
        predictions = self.sigmoid(model)
        prediction_class = np.where(predictions > 0.5, 1, 0)

        # Reshape to the square matrix representing causality
        predicted_causal_matrix = np.reshape(prediction_class, (n_unique_alarms, n_unique_alarms))
        
        return predicted_causal_matrix


In [ ]:
def create_features(dataset_list):
    new_dataset_list = []
    for alarm, causal in dataset_list:
        
        alarm['duration'] = alarm['end_timestamp'] - alarm['start_timestamp']
        alarm_one_hot = pd.get_dummies(alarm['alarm_id'], prefix='alarm')
        device_one_hot = pd.get_dummies(alarm['device_id'], prefix='device')

        # Time-windowed aggregations - Likely unnecessary as handled in training.
        alarm['count_last_5min'] = alarm.groupby('device_id')['alarm_id'].transform(lambda x: x.rolling(window=300, min_periods=0).count()).astype(int)
        alarm['count_last_3min'] = alarm.groupby('device_id')['alarm_id'].transform(lambda x: x.rolling(window=180, min_periods=0).count()).astype(int)

        # Alarm sequences
        alarm['last_alarm'] = alarm.groupby('device_id')['alarm_id'].shift().fillna(-1).astype(int)  # Fill NaN with -1

        # Elapsed time
        alarm['time_since_last_alarm'] = alarm.groupby('device_id')['start_timestamp'].diff().fillna(-1).astype(int)  # Fill NaN with -1
        alarm = pd.concat([alarm, alarm_one_hot, device_one_hot], axis=1)
        flattened_causal = causal.flatten()
        new_dataset_list.append((alarm, flattened_causal))
        
    return new_dataset_list

In [ ]:

def evaluate_model(trained_model, X_test, y_test):
    # Calculate metrics
    return accuracy, precision, recall


In [ ]:
def fine_tune_model(trained_model):
    # Hyperparameter tuning
    return fine_tuned_model


In [ ]:
def deploy_model(fine_tuned_model, new_data):
    # Make predictions
    return root_causes


In [ ]:
dataset_list = [(alarm1, causal1), (alarm2, causal2), (alarm3, causal3), (alarm4, causal4)]
dataset_list = create_features(dataset_list)

In [ ]:
counter = 0
dataset_dict = {}

for i in range(4):
    for j in range(4):
        if i != j:
            X_train, y_train = dataset_list[i]
            X_test, y_test = dataset_list[j]
            
            dataset_dict[f'dataset_{counter}'] = {
                'X_train': X_train,
                'y_train': y_train,
                'X_test': X_test,
                'y_test': y_test
            }
            
            counter += 1



In [ ]:
def dont_run():# Create an instance of the MarkovModel
    markov_model = MarkovModel(num_nodes)

    # Train the model using training data
    X_train, y_train = dataset_dict['dataset_0']['X_train'], dataset_dict['dataset_0']['y_train']
    markov_model.train(X_train, y_train)

    # Make predictions using test data
    X_test = dataset_dict['dataset_1']['X_test']
    predictions = markov_model.predict(X_test)

In [ ]:
alarm1


In [ ]:
for num, row in enumerate(causal1):
    print(num, row)


In [ ]:
for num, row in enumerate(topology1):
    print(num, row)